## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saldanha,ZA,-33.0117,17.9442,57.25,97,15,13.65,few clouds
1,1,Cahors,FR,44.4167,1.3333,47.70,82,0,5.68,clear sky
2,2,Cape Town,ZA,-33.9258,18.4232,59.09,73,40,16.11,scattered clouds
3,3,Jucurutu,BR,-6.0339,-37.0203,85.60,56,89,10.49,overcast clouds
4,4,Port Elizabeth,ZA,-33.9180,25.5701,67.10,82,100,8.19,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Salinas,US,36.6777,-121.6555,76.01,60,75,14.97,broken clouds
13,13,Georgetown,MY,5.4112,100.3354,80.55,87,40,1.99,scattered clouds
16,16,Vaini,TO,-21.2000,-175.2000,78.96,83,20,9.22,few clouds
18,18,Garissa,KE,-0.4536,39.6401,76.51,72,13,17.00,few clouds
24,24,Kavieng,PG,-2.5744,150.7967,82.26,74,35,4.25,scattered clouds
26,26,Owando,CG,-0.4819,15.8999,76.53,79,54,1.70,broken clouds
27,27,Touros,BR,-5.1989,-35.4608,81.50,81,72,15.73,broken clouds
32,32,San Jose,US,37.3394,-121.8950,75.78,46,75,10.00,broken clouds
39,39,Jasper,US,37.2001,-94.3502,81.00,53,1,1.01,clear sky
43,43,Nanakuli,US,21.3906,-158.1547,84.15,73,20,14.97,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             192
City                   192
Country                190
Lat                    192
Lng                    192
Max Temp               192
Humidity               192
Cloudiness             192
Wind Speed             192
Current Description    192
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.replace(' ', np.nan)                   
preferred_cities_df.dropna()        

preferred_cities_df.count()

Unnamed: 0             192
City                   192
Country                190
Lat                    192
Lng                    192
Max Temp               192
Humidity               192
Cloudiness             192
Wind Speed             192
Current Description    192
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Salinas,US,76.01,broken clouds,36.6777,-121.6555,
13,Georgetown,MY,80.55,scattered clouds,5.4112,100.3354,
16,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,
18,Garissa,KE,76.51,few clouds,-0.4536,39.6401,
24,Kavieng,PG,82.26,scattered clouds,-2.5744,150.7967,
26,Owando,CG,76.53,broken clouds,-0.4819,15.8999,
27,Touros,BR,81.50,broken clouds,-5.1989,-35.4608,
32,San Jose,US,75.78,broken clouds,37.3394,-121.8950,
39,Jasper,US,81.00,clear sky,37.2001,-94.3502,
43,Nanakuli,US,84.15,few clouds,21.3906,-158.1547,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

    
           

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Salinas,US,76.01,broken clouds,36.6777,-121.6555,Laurel Inn & Conference Center
13,Georgetown,MY,80.55,scattered clouds,5.4112,100.3354,Cititel Penang
16,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,Keleti Beach Resort
18,Garissa,KE,76.51,few clouds,-0.4536,39.6401,Sharif Plaza Hotel
24,Kavieng,PG,82.26,scattered clouds,-2.5744,150.7967,Nusa Island Retreat
26,Owando,CG,76.53,broken clouds,-0.4819,15.8999,Exaunel Hotel Owando
27,Touros,BR,81.50,broken clouds,-5.1989,-35.4608,INN NEW HORIZON
32,San Jose,US,75.78,broken clouds,37.3394,-121.8950,The Westin San Jose
39,Jasper,US,81.00,clear sky,37.2001,-94.3502,Guest House Motel
43,Nanakuli,US,84.15,few clouds,21.3906,-158.1547,Camp Pālehua


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file)


In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]



In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))